In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import
import pandas as pd
from pprint import pprint
from zen_garden.postprocess.results import Results
from pathlib import Path
from os import chdir

# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [3]:
opt = Results('../data/outputs/05_CCTS_WTE_small_01_cost_optimal')
n1= Results('../data/outputs/05_CCTS_WTE_small_02_n1')
expected_not_stored = Results('../data/outputs/05_CCTS_WTE_small_03_n1_expected_carbon_not_stored')
n1_adjusted = Results('../data/outputs/05_CCTS_WTE_small_04_n1_adjusted')

list = [opt, n1, expected_not_stored, n1_adjusted]

In [29]:
import pandas as pd

# List of technologies to keep
tech_filter = ['truck_batch', 'train_batch', 'flowline']

# Initialize an empty DataFrame
df = pd.DataFrame()

for i, scenario in enumerate(list):  # Assuming 'list' is your list of scenarios
    # Assuming scenario.get_df() saves a CSV file named 'capacity.csv'
    scenario.get_df("capacity", to_csv="capacity.csv")
    
    # Load the CSV file and filter by technology
    capacity = pd.read_csv('capacity.csv')
    capacity = capacity[capacity['technology'].isin(tech_filter)]
    
    # If it's the first scenario, set the 'node' and 'technology' columns
    if i == 0:
        df['edges'] = capacity['location']
        df['technology'] = capacity['technology']
        
    # Add the capacity values as a new column, named for the scenario
    df[f'scenario_{scenario.name}'] = capacity['capacity'].values  # Using .values to avoid index mismatch

# Remove rows where all capacity values across scenarios are 0
df = df.loc[~(df.filter(like='scenario').eq(0).all(axis=1))]

# Save the DataFrame to a CSV file
df.to_csv('tables/aggregated_capacity.csv', index=False)


In [30]:
import pandas as pd

# List of technologies to keep
tech_filter = ['truck_batch', 'train_batch', 'flowline']

# Initialize an empty DataFrame
df = pd.DataFrame()

for i, scenario in enumerate(list):  # Assuming 'list' is your list of scenarios
    # Assuming scenario.get_df() saves a CSV file named 'capacity.csv'
    scenario.get_df("capacity", to_csv="capacity.csv")
    
    # Load the CSV file and filter by technology
    capacity = pd.read_csv('capacity.csv')
    capacity = capacity[capacity['technology'].isin(tech_filter)]
    
    # If it's the first scenario, set the 'node' and 'technology' columns
    if i == 0:
        df['edge'] = capacity['location']
        df['technology'] = capacity['technology']
        
    # Add the capacity values as a new column, named for the scenario
    df[f'scenario_{i+1}'] = capacity['capacity'].values  # Using .values to avoid index mismatch

# Remove rows where all capacity values across scenarios are 0
df = df.loc[~(df.filter(like='scenario').eq(0).all(axis=1))]

# Calculate the difference to scenario 1 for subsequent scenarios
for scenario_col in df.columns[3:]:  # Start from 3rd column as first two are 'node' and 'technology'
    df[scenario_col] = df[scenario_col] - df['scenario_1']

# Save the DataFrame to a CSV file
df.to_csv('tables/aggregated_capacity_difference.csv', index=False)
